In [ ]:
from PIL import Image
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def fen_to_color(string):
  out = {}
  letters = "ABCDEFGH"
  for i in range(8):
    for j in range(1,9):
      out[letters[i] + str(j)] = 0
  string = string.split('/')
  new_string = []
  for s in string:
    for d in s:
      if d.isnumeric():
        ix = s.index(d)
        for i in range(int(d)-1):
          s = s[0:ix] + '1' + s[ix:]
    new_string.append(s)
  for i in range(8, 0, -1):
    for j in range(8):
      if new_string[8-i][j].islower():
        out[letters[j] + str(i)] = 1 #orange
      elif new_string[8-i][j].isupper():
        out[letters[j] + str(i)] = 2 #blue
  return out

def fen_to_dict(string):

  name_to_num = {
      'p' : 1,
      'b' : 2,
      'n' : 3,
      'r' : 4,
      'q' : 5,
      'k' : 6
  }

  out = {}
  letters = "ABCDEFGH"
  for i in range(8):
    for j in range(1,9):
      out[letters[i] + str(j)] = 0
  string = string.split('/')
  new_string = []
  for s in string:
    for d in s:
      if d.isnumeric():
        ix = s.index(d)
        for i in range(int(d)-1):
          s = s[0:ix] + '1' + s[ix:]
    new_string.append(s)
  for i in range(8, 0, -1):
    for j in range(8):
      if new_string[8-i][j].isnumeric():
        out[letters[j] + str(i)] = 0
      else:
        out[letters[j] + str(i)] = name_to_num[new_string[8-i][j].lower()]

  return out

def cropped_board_to_tiles(img):
    CHESS_TILES = {}
    num_rows = img.shape[0]
    num_cols = img.shape[1]
    rows_per_tile = num_rows//8
    cols_per_tile = num_cols//8
    letters = "ABCDEFGH"
    for i in range(8, 0, -1):
        for j in range(8):
            CHESS_TILES[letters[j] + str(i)] = img[(8 - i) * rows_per_tile : (9 - i) * rows_per_tile, j * cols_per_tile : (j + 1) * cols_per_tile] 
    return CHESS_TILES

def img_to_file(img, base_directory=None):
    name = 'f' + "".join(map(str, np.random.permutation(10).tolist())) + ".jpg"
    if base_directory is None:
        wpath = name
    else: 
        wpath = base_directory + name
    cv2.imwrite(wpath, img)
    return name

def board_to_64_files(img, positions, colors, base_directory=None):
    CHESS_TILES = {}
    dict = cropped_board_to_tiles(img)
    for key in dict.keys():
        CHESS_TILES[key] = (img_to_file(dict[key], base_directory), positions[key], colors[key])
    return CHESS_TILES

In [ ]:
# Example Code:

# img = image of board
img = cv2.imread("/content/drive/MyDrive/turk_data/rasbperry_turk/raw/1/0/1481228945.jpg")
cv2_imshow(img)

# fen_string is a string that is a fen
fen_string = '2nrQ2N/2P1p1PP/2p4B/kPp3K1/P5Qq/qnRrPPpb/B1N2p1p/P1pRp2b'

# given a fen it generates a dict of colors of positions of board
colors = fen_to_color(fen_string)


# given a fen it generates a dict of positions of board
positions = fen_to_dict(fen_string)

# breaks the board into 64 tiles and annotates them with the dict generated from fen
# writes the 64 images
tiles = board_to_64_files(img, positions, colors)

# creates a pandas dataframe
data_frame = pd.DataFrame(tiles)
data_frame = data_frame.transpose()

# creates a csv
csv_file = data_frame.to_csv('my_csv.csv', header=False, index=False)

# FEN For Picture where top right is a orange rook

# 5P1r/1q1Rp1P1/3r1P2/qqppk1R1/1b1k1n2/B1r2n1B/N2B3R/2b1bN2



# FEN For Picture where all corners are empty

# 1b1b1qP1/1N2r2R/2BP1k2/1k1q1RB1/r1npq1r1/1B6/2pnP1NR/1b2p3